# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv('../output_data/cities.csv')
vacation_df
vacation = vacation_df.dropna()
humidity = vacation["Humidity"].fillna(value = vacation["Humidity"].mean() )


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
vacation

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


In [4]:
locations = vacation[["Lat", "Lng"]]

In [6]:
fig = gmaps.figure( center= (46, -5), zoom_level= 2 )
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
narrowed_list = vacation.loc[(vacation["Max Temp"]>70) &  (vacation["Max Temp"]<80) & (vacation["Wind Speed"]<10) & (vacation["Cloudiness"]== 0)].dropna()

In [8]:
#coordinates_df = narrowed_list[["Lat", "Lng"]]
#coordinates_df["coordinate"]=coordinates_df[["Lat","Lng"]].apply(tuple, axis =1)
#coordinates = coordinates_df["coordinate"]
#coordinates_df
narrowed_list.reset_index(inplace = True)
narrowed_list

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
5,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
6,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [11]:

hotels=[]
for i in range(len(narrowed_list)):
    lat = narrowed_list.loc[i]['Lat']
    lng = narrowed_list.loc[i]['Lng']
    params = {"location": f"{lat},{lng}",
             "radius" : 5000,
             "key": g_key,
             "type": "hotel"}
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response= requests.get(base_url,params=params)
    hotel_names = response.json()
    hotels.append(hotel_names["results"][0]["name"])


hotels
narrowed_list["Hotel Name"]=hotels
narrowed_list 

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Capitólio
1,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
2,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Morondava
3,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Jagalbet
4,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Nador
5,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mogok
6,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Birjand


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = narrowed_list[["City_ID", "City", "Country", "Date", "Lat", "Lng", "Max Temp", "Hotel Name"]]

In [13]:
hotel_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Hotel Name
0,70,ponta do sol,BR,1558378767,-20.63,-46.00,73.68,Capitólio
1,88,cidreira,BR,1558378770,-30.17,-50.22,76.56,CIDREIRA
2,138,morondava,MG,1558378780,-20.30,44.28,79.08,Morondava
3,139,vaini,IN,1558378780,15.34,74.49,78.90,Jagalbet
4,176,nador,MA,1558378499,35.17,-2.93,75.20,Nador
5,297,mogok,MM,1558378844,22.92,96.51,72.24,Mogok
6,444,birjand,IR,1558378880,32.86,59.22,71.60,Birjand


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map

markers= gmaps.marker_layer(locations)
# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))